In [1]:
# Importando as bibliotecas que serão utilizadas

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from yellowbrick.classifier import ConfusionMatrix

In [2]:
# Importando a base de dados que será utilizada

credito = pd.read_csv(r'D:\Lucas\Estudos\Ciência de dados\31.Prática em Python\dados\Credit.csv', sep=',')
print(credito.shape)
credito.head()

(1000, 21)


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [3]:
# Formato de matriz

previsores = credito.iloc[:, 0:20].values
classe = credito.iloc[:, 20].values

In [4]:
# Transformação dos atributos categóricos em atributos numéricos, passando o índice de cada coluna categórica
# É necessário criar um objeto para cada atributo categórico, pois na sequência será executado o processo de enconding novamente para o registro do teste
# Se forem utilizados objetos diferentes, o número de atribuído a cada valor poderá ser diferente, o que deixará o teste inconsistente

# Montei um laço onde é identificado quais colunas tem dados não numéricos para serem transformados, e cria um objeto Label Encoder para cada coluna

for i in range(len(previsores[0])):
    if (type(previsores[0][i]) != 'int') and (type(previsores[0][i]) != 'float'):
        globals().__setitem__('labelencoder'+str(i), LabelEncoder())
        previsores[:, i] = globals()['labelencoder'+str(i)].fit_transform(previsores[:, i])
        
previsores

array([[2, 2, 1, ..., 0, 1, 1],
       [1, 29, 3, ..., 0, 0, 1],
       [0, 8, 1, ..., 1, 0, 1],
       ...,
       [0, 8, 3, ..., 0, 0, 1],
       [2, 27, 3, ..., 0, 1, 1],
       [1, 27, 1, ..., 0, 0, 1]], dtype=object)

In [5]:
# Dividindo a base em treinamento e teste

X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(previsores, classe, test_size = 0.3, random_state = 0)

In [6]:
#Método de classificação por vetor de suporte

svm = SVC()
svm.fit(X_treinamento, Y_treinamento)

SVC()

In [7]:
# Previsões dos dados de teste

previsoes = svm.predict(X_teste)
previsoes

array(['good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
      

In [8]:
# Taxa de acertos

taxa_acerto = accuracy_score(Y_teste, previsoes)
taxa_acerto

0.7133333333333334

In [9]:
# Utiilizando o algoritmo ExtraTreesClassifier para extrair as características mais importantes

forest = ExtraTreesClassifier()
forest.fit(X_treinamento, Y_treinamento)

importancias = forest.feature_importances_

importancias

array([0.10950888, 0.08494936, 0.06145589, 0.05821328, 0.07894012,
       0.0473892 , 0.05296415, 0.05025196, 0.04450351, 0.02718701,
       0.05106683, 0.05086115, 0.06495303, 0.03586694, 0.03646006,
       0.03588729, 0.04448553, 0.0229291 , 0.03404487, 0.00808183])

In [10]:
#Crianção de nova base de dados utilizando somente os atributos mais importantes

X_treinamento2 = X_treinamento[:, [0,1,2,3,4]]
X_teste2 = X_teste[:, [0,1,2,3,4]]                

In [12]:
#Criaindo outro modelo com a base de dados reduzida, treinando, obtendo as previsões e verificando a taxa de acertos

svm2 = SVC()
svm2.fit(X_treinamento2, Y_treinamento)
previsoes2 = svm2.predict(X_teste2)
taxa_acerto = accuracy_score(Y_teste, previsoes2)
taxa_acerto

0.7133333333333334